# Sudoku Solver - Project 1

This notebook implements and compares different algorithms for solving Sudoku puzzles:
- Backtracking (BT) - Systematic search with constraint propagation
- Forward Checking (FC) - Backtracking with forward constraint checking
- Arc Consistency (AC3) - Advanced constraint propagation algorithm
- Simulated Annealing (SA) - Stochastic optimization with cooling schedule
- Genetic Algorithm (GA) - Evolutionary approach with population-based search

All algorithms are implemented as simple functions with minimal object-oriented approach.


In [41]:
# =============================================================================
# CONFIGURATION PARAMETERS
# =============================================================================

# Project identification parameters
GROUP_ID = '21'  # Group identifier for output file naming
ALGORITHM = 'ga'  # Algorithm to use: 'bt' (simple backtracking), 'fc' (backtracking with FC), 
                  # 'ac3' (backtracking with AC3), 'sa' (simulated annealing), 'ga' (genetic algorithm)
PUZZLE_TYPE = 'easy'  # Difficulty level: 'easy', 'medium', 'hard', 'extreme'
PUZZLE_PATH = 'puzzles/Easy-P4.txt'  # Path to the puzzle file to solve

# =============================================================================
# SIMULATED ANNEALING (SA) PARAMETERS
# =============================================================================
# Using alpha-based cooling schedule as specified in the algorithm description
SA_T0 = 1000.0  # Initial temperature - high value allows exploration of poor solutions initially
SA_ALPHA = 0.995  # Cooling rate (alpha) - controls how quickly temperature decreases
SA_MAX_STEPS = 1000000  # Maximum number of iterations before stopping

# =============================================================================
# GENETIC ALGORITHM (GA) PARAMETERS - OPTIMIZED BY AUTOMATED TUNING
# =============================================================================
# Best configuration found through hyperparameter optimization:
# Population=200, Crossover=0.77, Mutation=0.043, Tournament k=5
# Performance: Runtime=7.5s, Decisions=42340, Success rate=100%
GA_POP = 300  # Population size - number of individuals in each generation
GA_CROSSOVER_RATE = 0.85  # Probability of crossover between two parents
GA_MUTATION_RATE = 0.06  # Probability of mutation for each individual
GA_MAX_GENERATIONS = 2000  # Maximum generations before restart (increased for harder puzzles)
GA_TOURNAMENT_K = 7  # Tournament size for selection (k individuals compete)

# =============================================================================
# GENERAL PARAMETERS
# =============================================================================
SEED = 123  # Random seed for reproducibility of results
MAX_STEPS = 10000  # Maximum steps for algorithms that use step counting


In [42]:
# =============================================================================
# LIBRARY IMPORTS AND SEED INITIALIZATION
# =============================================================================

# Import required libraries for Sudoku solving algorithms
import numpy as np  # For efficient array operations and matrix manipulation
import re  # For regular expressions used in puzzle file parsing
import copy  # For deep copying Sudoku grids during algorithm execution
import random  # For random number generation in SA and GA algorithms
import math  # For mathematical functions like exp() used in SA cooling schedule
from collections import deque  # For queue operations used in AC3 algorithm
import os  # For file path operations and output file naming

# =============================================================================
# RANDOM SEED INITIALIZATION
# =============================================================================
# Set seed for reproducibility - ensures same random sequence across runs
random.seed(SEED)  # Initialize Python's random module with fixed seed
np.random.seed(SEED)  # Initialize NumPy's random module with same seed


In [43]:
# =============================================================================
# DECISION COUNTER - GLOBAL VARIABLE APPROACH
# =============================================================================
# Simple global variable to track the number of decisions made by algorithms
# This is used to measure algorithm performance and compare efficiency

decisions = 0  # Global counter for tracking algorithm decisions

def inc_decisions(n=1):
    """
    Increment the global decision counter by n (default 1)
    
    Args:
        n (int): Number of decisions to add to counter (default: 1)
    """
    global decisions  # Access the global decisions variable
    decisions += n  # Add n to the current decision count

def reset_decisions():
    """
    Reset the global decision counter to zero
    Called at the start of each algorithm run to ensure clean counting
    """
    global decisions  # Access the global decisions variable
    decisions = 0  # Set decision counter back to zero


In [44]:
# =============================================================================
# CORE SUDOKU FUNCTIONS
# =============================================================================
# Fundamental functions for reading, writing, validating, and manipulating Sudoku puzzles

def read_puzzle_txt(path):
    """
    Read 9x9 Sudoku puzzle from text file with BOM support
    
    Args:
        path (str): Path to the puzzle file
        
    Returns:
        np.ndarray: 9x9 numpy array representing the Sudoku grid (0 = empty cell)
        
    Raises:
        FileNotFoundError: If the file doesn't exist
        ValueError: If the file format is invalid
    """
    try:
        # Open file with UTF-8 BOM support to handle files with byte order marks
        with open(path, 'r', encoding='utf-8-sig') as f:
            # Read all lines, strip whitespace, and filter out empty lines
            lines = [line.strip() for line in f.readlines() if line.strip()]
        
        # Validate that we have exactly 9 lines (rows)
        if len(lines) != 9:
            raise ValueError(f"Expected 9 lines, got {len(lines)}")
        
        grid = []  # Initialize empty grid list
        for i, line in enumerate(lines):  # Process each line (row)
            # Check if line contains comma-separated values
            if ',' in line:
                # Split by comma and strip whitespace from each value
                values = [v.strip() for v in line.split(',')]
                # Validate that we have exactly 9 values per row
                if len(values) != 9:
                    raise ValueError(f"Line {i+1} has {len(values)} values, expected 9")
            else:
                # Check if line contains only digits and question marks (9 characters)
                if not re.match(r'^[0-9?]{9}$', line):
                    raise ValueError(f"Line {i+1} contains invalid characters: {line}")
                # Convert string to list of characters
                values = list(line)
            
            row = []  # Initialize empty row list
            for value in values:  # Process each cell value
                if value == '?':  # Empty cell represented by '?'
                    row.append(0)  # Convert to 0 for internal representation
                else:
                    row.append(int(value))  # Convert digit string to integer
            grid.append(row)  # Add completed row to grid
        
        # Convert grid list to numpy array with integer data type
        return np.array(grid, dtype=int)
    
    except FileNotFoundError:
        raise FileNotFoundError(f"Input file not found: {path}")
    except Exception as e:
        raise ValueError(f"Error reading puzzle file: {e}")

def write_puzzle_txt(path, grid):
    """
    Write 9x9 Sudoku puzzle to text file in comma-separated format
    
    Args:
        path (str): Path where to save the puzzle file
        grid (np.ndarray): 9x9 numpy array representing the Sudoku grid
        
    Raises:
        ValueError: If grid is not 9x9
    """
    # Validate grid dimensions
    if grid.shape != (9, 9):
        raise ValueError(f"Grid must be 9x9, got {grid.shape}")
    
    # Open file for writing
    with open(path, 'w') as f:
        for row in grid:  # Process each row
            # Convert each cell: 0 becomes '?', other numbers become strings
            line = ','.join('?' if cell == 0 else str(cell) for cell in row)
            f.write(line + '\n')  # Write row with newline

def sudoku_from_grid(grid):
    """
    Create Sudoku object from 9x9 grid
    
    Args:
        grid (np.ndarray): 9x9 numpy array representing the Sudoku grid
        
    Returns:
        dict: Sudoku object with 'grid' and 'givens_mask' keys
        
    Raises:
        ValueError: If grid is invalid
    """
    # Validate grid is a 9x9 numpy array
    if not isinstance(grid, np.ndarray) or grid.shape != (9, 9):
        raise ValueError("Grid must be 9x9 numpy array")
    
    # Validate all values are in range 0-9
    if not np.all((grid >= 0) & (grid <= 9)):
        raise ValueError("Grid values must be in range 0-9")
    
    # Create mask indicating which cells are given (non-zero)
    givens_mask = grid > 0
    
    # Return Sudoku object as dictionary
    return {
        'grid': grid.copy(),  # Copy to avoid modifying original
        'givens_mask': givens_mask  # Boolean mask of given cells
    }

def is_solved(sud):
    """
    Check if Sudoku puzzle is completely solved
    
    Args:
        sud (dict): Sudoku object with 'grid' key
        
    Returns:
        bool: True if puzzle is completely solved, False otherwise
    """
    grid = sud['grid']  # Extract grid from Sudoku object
    
    # Check if any cells are still empty (0)
    if np.any(grid == 0):
        return False
    
    # Check each row for validity
    for row in grid:
        # Row is valid if it has 9 unique values, all in range 1-9
        if len(set(row)) != 9 or not all(1 <= x <= 9 for x in row):
            return False
    
    # Check each column for validity
    for col in grid.T:  # Transpose to get columns
        # Column is valid if it has 9 unique values, all in range 1-9
        if len(set(col)) != 9 or not all(1 <= x <= 9 for x in col):
            return False
    
    # Check each 3x3 box for validity
    for i in range(0, 9, 3):  # Iterate through box rows
        for j in range(0, 9, 3):  # Iterate through box columns
            # Extract 3x3 box and flatten to 1D array
            box = grid[i:i+3, j:j+3].flatten()
            # Box is valid if it has 9 unique values, all in range 1-9
            if len(set(box)) != 9 or not all(1 <= x <= 9 for x in box):
                return False
    
    return True  # All constraints satisfied

def total_conflicts(sud):
    """
    Count total number of constraint violations in the current state
    
    Args:
        sud (dict): Sudoku object with 'grid' key
        
    Returns:
        int: Total number of constraint violations (lower is better)
    """
    grid = sud['grid']  # Extract grid from Sudoku object
    conflicts = 0  # Initialize conflict counter
    
    # Check rows for conflicts
    for r in range(9):  # For each row
        # Get all non-zero values in this row
        row_values = [grid[r, c] for c in range(9) if grid[r, c] != 0]
        # Conflicts = total values - unique values (duplicates)
        conflicts += len(row_values) - len(set(row_values))
    
    # Check columns for conflicts
    for c in range(9):  # For each column
        # Get all non-zero values in this column
        col_values = [grid[r, c] for r in range(9) if grid[r, c] != 0]
        # Conflicts = total values - unique values (duplicates)
        conflicts += len(col_values) - len(set(col_values))
    
    # Check 3x3 boxes for conflicts
    for i in range(0, 9, 3):  # For each box row
        for j in range(0, 9, 3):  # For each box column
            box_values = []  # Initialize box values list
            # Collect all non-zero values in this 3x3 box
            for r in range(i, i+3):
                for c in range(j, j+3):
                    if grid[r, c] != 0:
                        box_values.append(grid[r, c])
            # Conflicts = total values - unique values (duplicates)
            conflicts += len(box_values) - len(set(box_values))
    
    return conflicts  # Return total conflict count

def peers(r, c):
    """
    Get all peer cells (same row, column, or 3x3 box) excluding (r,c)
    
    Args:
        r (int): Row index (0-8)
        c (int): Column index (0-8)
        
    Returns:
        set: Set of (row, col) tuples representing peer cell coordinates
    """
    peer_set = set()  # Initialize set to store peer coordinates
    
    # Add all cells in the same row (excluding current cell)
    for cc in range(9):
        if cc != c:  # Skip the current cell
            peer_set.add((r, cc))
    
    # Add all cells in the same column (excluding current cell)
    for rr in range(9):
        if rr != r:  # Skip the current cell
            peer_set.add((rr, c))
    
    # Add all cells in the same 3x3 box (excluding current cell)
    box_r = (r // 3) * 3  # Calculate box start row
    box_c = (c // 3) * 3  # Calculate box start column
    for rr in range(box_r, box_r + 3):  # Iterate through box rows
        for cc in range(box_c, box_c + 3):  # Iterate through box columns
            if (rr, cc) != (r, c):  # Skip the current cell
                peer_set.add((rr, cc))
    
    return peer_set  # Return set of peer coordinates

def box_coords(br, bc):
    """
    Get coordinates of all cells in a specific 3x3 box
    
    Args:
        br (int): Box row index (0-2)
        bc (int): Box column index (0-2)
        
    Returns:
        list: List of (row, col) tuples for all cells in the specified box
    """
    coords = []  # Initialize list to store coordinates
    # Calculate actual grid coordinates from box coordinates
    for r in range(br*3, (br+1)*3):  # For each row in the box
        for c in range(bc*3, (bc+1)*3):  # For each column in the box
            coords.append((r, c))  # Add coordinate to list
    return coords  # Return list of coordinates


In [45]:
# =============================================================================
# BACKTRACKING ALGORITHMS (BT, FC, AC3)
# =============================================================================
# Implementation of constraint satisfaction algorithms for Sudoku solving

def sudoku_domain(sud, r, c):
    """
    Get possible values for cell (r,c) based on current state
    
    Args:
        sud (dict): Sudoku object with 'grid' key
        r (int): Row index (0-8)
        c (int): Column index (0-8)
        
    Returns:
        set: Set of valid values for the cell (1-9)
    """
    grid = sud['grid']  # Extract grid from Sudoku object
    
    # If cell already has a value, return only that value
    if grid[r, c] != 0:
        return {grid[r, c]}
    
    used_values = set()  # Initialize set to store used values
    # Check all peer cells (same row, column, or box)
    for pr, pc in peers(r, c):
        if grid[pr, pc] != 0:  # If peer cell has a value
            used_values.add(grid[pr, pc])  # Add to used values
    
    # Return all possible values (1-9) minus the used values
    return set(range(1, 10)) - used_values

def select_unassigned_variable(sud, use_mrv=True, use_deg=True):
    """
    Select unassigned variable using MRV and DEG heuristics
    
    Args:
        sud (dict): Sudoku object with 'grid' key
        use_mrv (bool): Whether to use Minimum Remaining Values heuristic
        use_deg (bool): Whether to use Degree heuristic as tiebreaker
        
    Returns:
        tuple or None: (row, col) of selected cell, or None if no unassigned cells
    """
    grid = sud['grid']  # Extract grid from Sudoku object
    # Find all unassigned cells (value = 0)
    unassigned = [(r, c) for r in range(9) for c in range(9) if grid[r, c] == 0]
    
    # If no unassigned cells, return None
    if not unassigned:
        return None
    
    # If MRV is disabled, return first unassigned cell
    if not use_mrv:
        return unassigned[0]
    
    min_domain_size = float('inf')  # Initialize minimum domain size
    candidates = []  # List to store cells with minimum domain size
    
    # Find cells with minimum remaining values (MRV heuristic)
    for r, c in unassigned:
        domain_size = len(sudoku_domain(sud, r, c))  # Calculate domain size
        if domain_size < min_domain_size:  # New minimum found
            min_domain_size = domain_size
            candidates = [(r, c)]  # Reset candidates list
        elif domain_size == min_domain_size:  # Tie for minimum
            candidates.append((r, c))  # Add to candidates
    
    # If only one candidate or DEG is disabled, return first candidate
    if len(candidates) == 1 or not use_deg:
        return candidates[0]
    
    # Use Degree heuristic as tiebreaker (most constrained variable)
    max_unassigned_peers = -1  # Initialize maximum unassigned peers
    best_candidate = candidates[0]  # Initialize best candidate
    
    # Find cell with most unassigned peers among candidates
    for r, c in candidates:
        # Count unassigned peer cells
        unassigned_peers = sum(1 for pr, pc in peers(r, c) if grid[pr, pc] == 0)
        if unassigned_peers > max_unassigned_peers:  # New maximum found
            max_unassigned_peers = unassigned_peers
            best_candidate = (r, c)  # Update best candidate
    
    return best_candidate  # Return most constrained variable

def remove_inconsistent_values(sud, xi, xj, domains):
    """
    Remove inconsistent values from domain of Xi based on constraint with Xj
    
    Args:
        sud (dict): Sudoku object with 'grid' key
        xi (tuple): First variable (row, col)
        xj (tuple): Second variable (row, col) 
        domains (dict): Dictionary mapping (row, col) to set of possible values
        
    Returns:
        bool: True if any values were removed, False otherwise
    """
    removed = False  # Track if any values were removed
    ri, ci = xi  # Unpack first variable coordinates
    rj, cj = xj  # Unpack second variable coordinates
    
    # Get current domain of Xi
    domain_xi = domains[xi].copy() if xi in domains else sudoku_domain(sud, ri, ci)
    
    # Check each value in Xi's domain
    for x in list(domain_xi):  # Use list to avoid modification during iteration
        # Check if there's any value in Xj's domain that satisfies the constraint
        domain_xj = domains[xj].copy() if xj in domains else sudoku_domain(sud, rj, cj)
        constraint_satisfied = False
        
        for y in domain_xj:
            # Check if (x, y) satisfies the constraint between Xi and Xj
            # For Sudoku, the constraint is that if Xi and Xj are peers, they cannot have the same value
            if (ri, ci) in peers(rj, cj):  # If they are peers
                if x != y:  # Different values satisfy the constraint
                    constraint_satisfied = True
                    break
            else:  # If they are not peers, any values satisfy the constraint
                constraint_satisfied = True
                break
        
        # If no value in Xj's domain allows (x, y) to satisfy the constraint
        if not constraint_satisfied:
            if xi in domains:
                domains[xi].discard(x)  # Remove x from Xi's domain
            removed = True
    
    return removed  # Return whether any values were removed

def ac3(sud):
    """
    Arc Consistency algorithm (AC3) for constraint propagation
    
    Args:
        sud (dict): Sudoku object with 'grid' key
        
    Returns:
        bool: True if AC3 succeeds, False if inconsistency detected
    """
    # Initialize domains for all unassigned variables
    domains = {}
    grid = sud['grid']
    
    # Create initial domains for all unassigned cells
    for r in range(9):
        for c in range(9):
            if grid[r, c] == 0:  # If cell is unassigned
                domains[(r, c)] = sudoku_domain(sud, r, c)
    
    # Initialize queue with all arcs (constraints between peer variables)
    queue = deque()
    
    # Add all arcs between unassigned variables and their peers
    for r in range(9):
        for c in range(9):
            if grid[r, c] == 0:  # If cell is unassigned
                for pr, pc in peers(r, c):  # For each peer
                    if grid[pr, pc] == 0:  # If peer is also unassigned
                        queue.append(((r, c), (pr, pc)))  # Add arc to queue
    
    # Main AC3 loop
    while queue:  # While queue is not empty
        xi, xj = queue.popleft()  # Remove first arc from queue
        
        # Try to remove inconsistent values from Xi's domain
        if remove_inconsistent_values(sud, xi, xj, domains):
            # If any values were removed from Xi's domain
            if len(domains[xi]) == 0:  # If Xi's domain is empty
                return False  # Inconsistency detected
            
            # Add arcs from Xi's neighbors back to queue
            ri, ci = xi  # Unpack Xi coordinates
            for neighbor in peers(ri, ci):  # For each neighbor of Xi
                if neighbor in domains:  # If neighbor is unassigned
                    queue.append((neighbor, xi))  # Add arc (neighbor, Xi) to queue
    
    return True  # AC3 succeeded

def solve_bt_with_ac3(sud, use_mrv=True, use_deg=True, use_lcv=True):
    """
    Solve Sudoku using backtracking with AC3 preprocessing
    
    Args:
        sud (dict): Sudoku object with 'grid' and 'givens_mask' keys
        use_mrv (bool): Whether to use Minimum Remaining Values heuristic
        use_deg (bool): Whether to use Degree heuristic as tiebreaker
        use_lcv (bool): Whether to use Least Constraining Value heuristic
        
    Returns:
        dict or None: Solved Sudoku object, or None if no solution exists
    """
    # Apply AC3 preprocessing
    if not ac3(sud):
        return None  # AC3 detected inconsistency
    
    grid = sud['grid'].copy()  # Create working copy of the grid
    
    def backtrack():
        """
        Recursive backtracking function with AC3 integration
        
        Returns:
            np.ndarray or None: Solved grid, or None if no solution found
        """
        # Check if all cells are assigned (no zeros)
        if not np.any(grid == 0):
            # Only check constraints when we have a complete assignment
            if is_solved({'grid': grid}):
                print(f"AC3 SOLUTION FOUND! Stopping at {decisions} decisions")
                return grid.copy()  # Return solution
            else:
                print(f"AC3 Complete but invalid assignment at {decisions} decisions")
                return None  # Complete but invalid assignment
        
        # Select next unassigned variable using heuristics
        var = select_unassigned_variable({'grid': grid}, use_mrv, use_deg)
        if var is None:  # No unassigned variables (shouldn't happen)
            return None
        
        r, c = var  # Unpack variable coordinates
        # Get ordered list of values to try (using LCV heuristic)
        values = order_domain_values({'grid': grid}, r, c, use_lcv)
        
        # Try each value in order
        for value in values:
            # Check if value is still valid (domain might have changed)
            if value in sudoku_domain({'grid': grid}, r, c):
                grid[r, c] = value  # Assign value
                inc_decisions(1)  # Increment decision counter
                
                # Recursively try to solve the rest
                result = backtrack()
                if result is not None:  # Solution found
                    return result
                
                grid[r, c] = 0  # Backtrack: unassign value
        
        return None  # No solution found with any value
    
    # Start backtracking search
    solution_grid = backtrack()
    if solution_grid is not None:  # Solution found
        return {'grid': solution_grid, 'givens_mask': sud['givens_mask']}
    else:  # No solution found
        return None

def solve_bt_simple(sud, use_mrv=True, use_deg=True, use_lcv=True):
    """
    Solve Sudoku using simple backtracking WITHOUT forward checking
    
    Implements the standard BACKTRACKING-SEARCH algorithm:
    function BACKTRACKING-SEARCH(csp)
        return RECURSIVE-BACKTRACKING({}, csp)
    
    Args:
        sud (dict): Sudoku object with 'grid' and 'givens_mask' keys
        use_mrv (bool): Whether to use Minimum Remaining Values heuristic
        use_deg (bool): Whether to use Degree heuristic as tiebreaker
        use_lcv (bool): Whether to use Least Constraining Value heuristic
        
    Returns:
        dict or None: Solved Sudoku object, or None if no solution exists
    """
    grid = sud['grid'].copy()  # Create working copy of the grid
    
    def is_assignment_complete(assignment):
        """Check if assignment is complete (all cells filled)"""
        return not np.any(grid == 0)
    
    def select_unassigned_variable_simple():
        """Select next unassigned variable using heuristics"""
        return select_unassigned_variable({'grid': grid}, use_mrv, use_deg)
    
    def order_domain_values_simple(var):
        """Order domain values for variable using LCV heuristic"""
        r, c = var
        return order_domain_values({'grid': grid}, r, c, use_lcv)
    
    def is_consistent(var, value):
        """Check if value is consistent with current assignment"""
        r, c = var
        # Check if value conflicts with any assigned peers
        for peer_r, peer_c in peers(r, c):
            if grid[peer_r, peer_c] == value:
                return False  # Conflict found
        return True  # No conflicts
    
    def recursive_backtracking(assignment):
        """
        RECURSIVE-BACKTRACKING(asg, csp)
        if asg is complete, then return asg
        var := SELECT-UNASSIGNED-VARIABLE(VARIABLES(csp), asg, csp)
        for each value in ORDER-DOMAIN-VALUES(var, asg, csp) do
            if value is consistent with asg using CONSTRAINTS(csp) then
                add {var = value} to asg
                result := RECURSIVE-BACKTRACKING(asg, csp)
                if result ≠ failure then return result
                remove {var = value} from asg
        return failure
        """
        # Progress reporting every 1000 decisions
        if decisions % 1000 == 0 and decisions > 0:
            empty_cells = np.sum(grid == 0)
            print(f"BT Progress: {decisions} decisions, {empty_cells} cells remaining")
        
        # if asg is complete, then return asg
        if is_assignment_complete(assignment):
            print(f"SOLUTION FOUND! Stopping at {decisions} decisions")
            return grid.copy()
        
        # var := SELECT-UNASSIGNED-VARIABLE(VARIABLES(csp), asg, csp)
        var = select_unassigned_variable_simple()
        if var is None:
            print("No unassigned variables found!")
            return None
        
        # Debug: Show what variable and values we're trying
        if decisions < 10:  # Only show first few for debugging
            values = order_domain_values_simple(var)
            print(f"Trying variable {var}, values: {values}")
        
        # for each value in ORDER-DOMAIN-VALUES(var, asg, csp) do
        for value in order_domain_values_simple(var):
            # if value is consistent with asg using CONSTRAINTS(csp) then
            if is_consistent(var, value):
                # add {var = value} to asg
                r, c = var
                grid[r, c] = value
                inc_decisions(1)
                
                # result := RECURSIVE-BACKTRACKING(asg, csp)
                result = recursive_backtracking(assignment)
                
                # if result ≠ failure then return result
                if result is not None:
                    return result
                
                # remove {var = value} from asg
                grid[r, c] = 0
        
        # return failure
        return None
    
    # BACKTRACKING-SEARCH(csp) - return RECURSIVE-BACKTRACKING({}, csp)
    solution_grid = recursive_backtracking({})
    if solution_grid is not None:
        return {'grid': solution_grid, 'givens_mask': sud['givens_mask']}
    else:
        return None

def order_domain_values(sud, r, c, use_lcv=True):
    """
    Order domain values using LCV (Least Constraining Value) heuristic
    
    Args:
        sud (dict): Sudoku object with 'grid' key
        r (int): Row index (0-8)
        c (int): Column index (0-8)
        use_lcv (bool): Whether to use LCV heuristic
        
    Returns:
        list: Ordered list of values to try (least constraining first)
    """
    domain = sudoku_domain(sud, r, c)  # Get valid values for the cell
    values = list(domain)  # Convert set to list
    
    # If LCV is disabled or only one value, return as-is
    if not use_lcv or len(values) <= 1:
        return values
    
    def count_constraints(value):
        """
        Count how many peer cells would be constrained by assigning this value
        
        Args:
            value (int): Value to test (1-9)
            
        Returns:
            int: Number of peer cells that would lose this value from their domain
        """
        count = 0  # Initialize constraint counter
        # Check all peer cells
        for pr, pc in peers(r, c):
            # If peer is unassigned and would lose this value from its domain
            if sud['grid'][pr, pc] == 0 and value in sudoku_domain(sud, pr, pc):
                count += 1  # Increment constraint count
        return count
    
    # Sort values by constraint count (ascending - least constraining first)
    return sorted(values, key=count_constraints)

def solve_bt(sud, use_mrv=True, use_deg=True, use_lcv=True):
    """
    Solve Sudoku using backtracking with forward checking
    
    Implements BACKTRACKING-SEARCH with forward checking:
    - Uses domain filtering to reduce search space
    - Checks constraints during assignment, not just at completion
    
    Args:
        sud (dict): Sudoku object with 'grid' and 'givens_mask' keys
        use_mrv (bool): Whether to use Minimum Remaining Values heuristic
        use_deg (bool): Whether to use Degree heuristic as tiebreaker
        use_lcv (bool): Whether to use Least Constraining Value heuristic
        
    Returns:
        dict or None: Solved Sudoku object, or None if no solution exists
    """
    grid = sud['grid'].copy()  # Create working copy of the grid
    
    def is_assignment_complete(assignment):
        """Check if assignment is complete (all cells filled)"""
        return not np.any(grid == 0)
    
    def select_unassigned_variable_fc():
        """Select next unassigned variable using heuristics"""
        return select_unassigned_variable({'grid': grid}, use_mrv, use_deg)
    
    def order_domain_values_fc(var):
        """Order domain values for variable using LCV heuristic with domain filtering"""
        r, c = var
        return order_domain_values({'grid': grid}, r, c, use_lcv)
    
    def is_consistent_fc(var, value):
        """Check if value is consistent using forward checking (domain filtering)"""
        r, c = var
        # Check if value is in the current domain (forward checking)
        return value in sudoku_domain({'grid': grid}, r, c)
    
    def recursive_backtracking(assignment):
        """
        RECURSIVE-BACKTRACKING with forward checking
        """
        # if asg is complete, then return asg
        if is_assignment_complete(assignment):
            print(f"FC SOLUTION FOUND! Stopping at {decisions} decisions")
            return grid.copy()
        
        # var := SELECT-UNASSIGNED-VARIABLE(VARIABLES(csp), asg, csp)
        var = select_unassigned_variable_fc()
        if var is None:
            return None
        
        # for each value in ORDER-DOMAIN-VALUES(var, asg, csp) do
        for value in order_domain_values_fc(var):
            # if value is consistent with asg using CONSTRAINTS(csp) then
            if is_consistent_fc(var, value):
                # add {var = value} to asg
                r, c = var
                grid[r, c] = value
                inc_decisions(1)
                
                # result := RECURSIVE-BACKTRACKING(asg, csp)
                result = recursive_backtracking(assignment)
                
                # if result ≠ failure then return result
                if result is not None:
                    return result
                
                # remove {var = value} from asg
                grid[r, c] = 0
        
        # return failure
        return None
    
    # BACKTRACKING-SEARCH(csp) - return RECURSIVE-BACKTRACKING({}, csp)
    solution_grid = recursive_backtracking({})
    if solution_grid is not None:
        return {'grid': solution_grid, 'givens_mask': sud['givens_mask']}
    else:
        return None


In [46]:
# =============================================================================
# SIMULATED ANNEALING (SA) - EXACT IMPLEMENTATION
# =============================================================================
# Stochastic optimization algorithm that uses temperature-based acceptance of worse solutions

def random_complete_assignment(sud):
    """
    Create random complete assignment respecting givens
    
    Args:
        sud (dict): Sudoku object with 'grid' and 'givens_mask' keys
        
    Returns:
        dict: Complete Sudoku assignment with all cells filled
    """
    C = sud['grid'].copy()  # Create working copy of the grid
    givens_mask = sud['givens_mask']  # Extract givens mask
    
    # Process each 3x3 box to ensure box constraints are satisfied
    for br in range(3):  # For each box row
        for bc in range(3):  # For each box column
            # Collect given values and empty cells in this box
            given_values = set()  # Set to store given values in this box
            cells = []  # List to store empty cell coordinates
            
            # Scan all cells in this 3x3 box
            for r in range(br*3, (br+1)*3):
                for c in range(bc*3, (bc+1)*3):
                    if givens_mask[r, c]:  # If cell is given
                        given_values.add(C[r, c])  # Add to given values
                    else:  # If cell is empty
                        cells.append((r, c))  # Add to empty cells list
            
            # Calculate missing values for this box (1-9 minus given values)
            missing = set(range(1, 10)) - given_values
            
            # Shuffle missing values to randomize assignment
            missing_list = list(missing)
            random.shuffle(missing_list)
            
            # Assign missing values to empty cells
            for i, (r, c) in enumerate(cells):
                if i < len(missing_list):  # If we have values to assign
                    C[r, c] = missing_list[i]  # Assign value to cell
    
    # Return complete assignment
    return {'grid': C, 'givens_mask': givens_mask}

def random_neighbor(asg):
    """
    Pick one variable at random; change its value to a different value from its domain
    
    Args:
        asg (dict): Current Sudoku assignment
        
    Returns:
        dict: New assignment with one cell changed
    """
    # Create copy of current assignment
    neighbor = {'grid': asg['grid'].copy(), 'givens_mask': asg['givens_mask']}
    
    # Find all non-given cells (cells that can be modified)
    non_given_cells = []
    for r in range(9):  # For each row
        for c in range(9):  # For each column
            if not asg['givens_mask'][r, c]:  # If cell is not given
                non_given_cells.append((r, c))  # Add to modifiable cells
    
    # If no modifiable cells, return unchanged assignment
    if not non_given_cells:
        return neighbor
    
    # Pick random cell to modify
    r, c = random.choice(non_given_cells)
    
    # Get current value of the cell
    current_value = asg['grid'][r, c]
    
    # Create domain (all values 1-9 except current value)
    domain = [v for v in range(1, 10) if v != current_value]
    
    # Pick new value from domain
    new_value = random.choice(domain)
    
    # Assign new value to the cell
    neighbor['grid'][r, c] = new_value
    inc_decisions(1)  # Increment decision counter
    
    return neighbor  # Return modified assignment

def smart_neighbor(asg):
    """
    Generate neighbor by swapping two non-given cells in same box (preserves box constraints)
    
    Args:
        asg (dict): Current Sudoku assignment
        
    Returns:
        dict: New assignment with two cells swapped within same box
    """
    # Create copy of current assignment
    neighbor = {'grid': asg['grid'].copy(), 'givens_mask': asg['givens_mask']}
    
    # Pick random 3x3 box
    br, bc = random.randint(0, 2), random.randint(0, 2)
    
    # Get non-given cells in this box
    cells = []  # List to store modifiable cells in this box
    for r in range(br*3, (br+1)*3):  # For each row in the box
        for c in range(bc*3, (bc+1)*3):  # For each column in the box
            if not asg['givens_mask'][r, c]:  # If cell is not given
                cells.append((r, c))  # Add to modifiable cells
    
    # If we have at least 2 modifiable cells, perform swap
    if len(cells) >= 2:
        # Pick two random cells from the box
        cell1, cell2 = random.sample(cells, 2)
        r1, c1 = cell1  # Unpack first cell coordinates
        r2, c2 = cell2  # Unpack second cell coordinates
        
        # Swap values between the two cells
        neighbor['grid'][r1, c1], neighbor['grid'][r2, c2] = neighbor['grid'][r2, c2], neighbor['grid'][r1, c1]
        inc_decisions(1)  # Increment decision counter
    
    return neighbor  # Return modified assignment

def solve_sa(sud, T0, alpha, max_steps):
    """
    Solve Sudoku using Simulated Annealing with hybrid neighbor generation
    
    Args:
        sud (dict): Sudoku object with 'grid' and 'givens_mask' keys
        T0 (float): Initial temperature
        alpha (float): Cooling rate (0 < alpha < 1)
        max_steps (int): Maximum number of iterations
        
    Returns:
        dict: Best Sudoku assignment found
    """
    # Create initial complete assignment
    asg = random_complete_assignment(sud)
    best = asg  # Track best solution found so far
    T = T0  # Initialize temperature
    
    # Main simulated annealing loop
    for step in range(max_steps):
        # Check if current assignment is solved (no conflicts)
        if total_conflicts(asg) == 0:
            return asg  # Return solution
        
        # Generate neighbor using hybrid strategy
        # Use smart neighbor (swap within box) 70% of the time, random neighbor 30%
        if random.random() < 0.7:
            nbr = smart_neighbor(asg)  # Smart neighbor preserves box constraints
        else:
            nbr = random_neighbor(asg)  # Random neighbor for exploration
        
        # Calculate change in objective function (conflicts)
        delta = total_conflicts(nbr) - total_conflicts(asg)
        
        # Accept neighbor if it's better (delta <= 0) or with probability based on temperature
        if delta <= 0 or random.random() < math.exp(-delta / max(T, 1e-10)):
            asg = nbr  # Accept the neighbor
            # Update best solution if current is better
            if total_conflicts(asg) < total_conflicts(best):
                best = asg
        
        T = alpha * T  # Cool down temperature (reduce by factor alpha)
    
    return best  # Return best solution found


In [47]:
# =============================================================================
# GENETIC ALGORITHM (GA) - EXACT IMPLEMENTATION
# =============================================================================
# Evolutionary algorithm that uses population-based search with selection, crossover, and mutation

def ga_fitness(individual):
    """
    Fitness function: higher is better
    
    Args:
        individual (dict): Sudoku individual with 'grid' key
        
    Returns:
        float: Fitness value (negative conflicts, so higher is better)
    """
    inc_decisions(1)  # Count each fitness evaluation as a decision
    return -total_conflicts(individual)  # Return negative conflicts (higher is better)

def ga_tournament_select(population, fitnesses, k):
    """
    Tournament selection: sample k individuals, return best
    
    Args:
        population (list): List of Sudoku individuals
        fitnesses (list): List of fitness values for each individual
        k (int): Tournament size (number of individuals to compete)
        
    Returns:
        dict: Copy of the best individual from the tournament
    """
    # Randomly select k individuals for tournament
    tournament_indices = random.sample(range(len(population)), k)
    # Find the individual with highest fitness in the tournament
    best_idx = max(tournament_indices, key=lambda i: fitnesses[i])
    # Return a copy of the best individual
    return {'grid': population[best_idx]['grid'].copy(), 'givens_mask': population[best_idx]['givens_mask']}

def ga_crossover(p1, p2):
    """
    Block-based crossover that preserves box constraints
    
    Args:
        p1 (dict): First parent Sudoku individual
        p2 (dict): Second parent Sudoku individual
        
    Returns:
        tuple: (child1, child2) - two offspring individuals
    """
    # Create copies of parents as starting points for children
    child1 = {'grid': p1['grid'].copy(), 'givens_mask': p1['givens_mask']}
    child2 = {'grid': p2['grid'].copy(), 'givens_mask': p2['givens_mask']}
    
    # Randomly choose which 3x3 blocks to copy from each parent
    for br in range(3):  # For each box row
        for bc in range(3):  # For each box column
            if random.random() < 0.5:  # 50% chance
                # Copy block from p1 to child1, p2 to child2
                for r in range(br*3, (br+1)*3):  # For each row in the box
                    for c in range(bc*3, (bc+1)*3):  # For each column in the box
                        child1['grid'][r, c] = p1['grid'][r, c]  # Copy from p1
                        child2['grid'][r, c] = p2['grid'][r, c]  # Copy from p2
            else:  # 50% chance
                # Copy block from p2 to child1, p1 to child2
                for r in range(br*3, (br+1)*3):  # For each row in the box
                    for c in range(bc*3, (bc+1)*3):  # For each column in the box
                        child1['grid'][r, c] = p2['grid'][r, c]  # Copy from p2
                        child2['grid'][r, c] = p1['grid'][r, c]  # Copy from p1
    
    # Repair givens - ensure given cells are preserved from original puzzle
    givens_mask = p1['givens_mask']  # Get givens mask
    for r in range(9):  # For each row
        for c in range(9):  # For each column
            if givens_mask[r, c]:  # If cell is given
                child1['grid'][r, c] = p1['grid'][r, c]  # Restore given value in child1
                child2['grid'][r, c] = p1['grid'][r, c]  # Restore given value in child2
    
    return child1, child2  # Return both offspring

def ga_mutate(individual, rate, sud):
    """
    Mutate using multiple strategies
    
    Args:
        individual (dict): Sudoku individual to mutate
        rate (float): Mutation probability (0-1)
        sud (dict): Original Sudoku puzzle (for givens mask)
        
    Returns:
        dict: Mutated individual
    """
    # Create copy of individual for mutation
    mutated = {'grid': individual['grid'].copy(), 'givens_mask': individual['givens_mask']}
    
    # Apply mutation with probability rate
    if random.random() < rate:
        # Strategy 1: Swap within same box (70% of the time)
        if random.random() < 0.7:
            # Pick random 3x3 box
            br, bc = random.randint(0, 2), random.randint(0, 2)
            
            # Get non-given cells in this box
            cells = []  # List to store modifiable cells
            for r in range(br*3, (br+1)*3):  # For each row in the box
                for c in range(bc*3, (bc+1)*3):  # For each column in the box
                    if not sud['givens_mask'][r, c]:  # If cell is not given
                        cells.append((r, c))  # Add to modifiable cells
            
            # If we have at least 2 modifiable cells, perform swap
            if len(cells) >= 2:
                # Pick two random cells and swap their values
                cell1, cell2 = random.sample(cells, 2)
                r1, c1 = cell1  # Unpack first cell coordinates
                r2, c2 = cell2  # Unpack second cell coordinates
                
                # Swap values between the two cells
                mutated['grid'][r1, c1], mutated['grid'][r2, c2] = mutated['grid'][r2, c2], mutated['grid'][r1, c1]
                inc_decisions(1)  # Increment decision counter
        
        # Strategy 2: Swap across boxes (30% of the time)
        else:
            # Find two non-given cells in different boxes and swap them
            non_given_cells = []  # List to store all modifiable cells
            for r in range(9):  # For each row
                for c in range(9):  # For each column
                    if not sud['givens_mask'][r, c]:  # If cell is not given
                        non_given_cells.append((r, c))  # Add to modifiable cells
            
            # If we have at least 2 modifiable cells, attempt cross-box swap
            if len(non_given_cells) >= 2:
                cell1, cell2 = random.sample(non_given_cells, 2)
                r1, c1 = cell1  # Unpack first cell coordinates
                r2, c2 = cell2  # Unpack second cell coordinates
                
                # Check if they're in different boxes
                box1 = (r1 // 3, c1 // 3)  # Calculate box coordinates for cell1
                box2 = (r2 // 3, c2 // 3)  # Calculate box coordinates for cell2
                
                # Only swap if they're in different boxes
                if box1 != box2:
                    mutated['grid'][r1, c1], mutated['grid'][r2, c2] = mutated['grid'][r2, c2], mutated['grid'][r1, c1]
                    inc_decisions(1)  # Increment decision counter
    
    return mutated  # Return mutated individual

def solve_ga(sud, pop_size, crossover_rate, mutation_rate, max_generations, tournament_k):
    """
    Solve Sudoku using Genetic Algorithm - runs until solution found with seed restart
    
    Args:
        sud (dict): Sudoku object with 'grid' and 'givens_mask' keys
        pop_size (int): Population size
        crossover_rate (float): Probability of crossover (0-1)
        mutation_rate (float): Probability of mutation (0-1)
        max_generations (int): Maximum generations per attempt
        tournament_k (int): Tournament size for selection
        
    Returns:
        dict: Solved Sudoku individual
    """
    attempt = 0  # Track number of attempts
    total_generations = 0  # Track total generations across all attempts
    
    # Infinite loop until solution is found
    while True:
        attempt += 1  # Increment attempt counter
        # Change seed every attempt for different random initialization
        new_seed = random.randint(1, 10000)
        random.seed(new_seed)  # Set new random seed
        np.random.seed(new_seed)  # Set new numpy random seed
        
        print(f"🔄 GA Attempt {attempt} (seed {new_seed})")
        
        # Initialize population with new seed
        population = [random_complete_assignment(sud) for _ in range(pop_size)]
        
        # Main genetic algorithm loop
        for gen in range(max_generations):
            # Check if any individual is solved
            for individual in population:
                if total_conflicts(individual) == 0:  # If no conflicts
                    print(f"🎉 GA SOLVED at generation {gen} (total: {total_generations + gen})!")
                    return individual  # Return solution
            
            # Calculate fitnesses for all individuals
            fitnesses = [ga_fitness(x) for x in population]
            
            # Progress reporting every 100 generations
            if gen % 100 == 0:
                best_fitness = max(fitnesses)  # Get best fitness
                best_conflicts = -best_fitness  # Convert to conflicts (lower is better)
                print(f"GA Generation {gen}: best_conflicts={best_conflicts}")
            
            # Create offspring population
            offspring = []  # List to store new generation
            
            # Generate offspring until population is full
            while len(offspring) < pop_size:
                # Selection: choose two parents using tournament selection
                p1 = ga_tournament_select(population, fitnesses, tournament_k)
                p2 = ga_tournament_select(population, fitnesses, tournament_k)
                
                # Crossover: create children from parents
                if random.random() < crossover_rate:  # If crossover occurs
                    c1, c2 = ga_crossover(p1, p2)  # Create two children
                else:  # If no crossover
                    c1 = {'grid': p1['grid'].copy(), 'givens_mask': p1['givens_mask']}  # Copy parent1
                    c2 = {'grid': p2['grid'].copy(), 'givens_mask': p2['givens_mask']}  # Copy parent2
                
                # Mutation: apply mutation to children
                c1 = ga_mutate(c1, mutation_rate, sud)  # Mutate child1
                c2 = ga_mutate(c2, mutation_rate, sud)  # Mutate child2
                
                # Add children to offspring population
                offspring.append(c1)  # Add first child
                if len(offspring) < pop_size:  # If population not full
                    offspring.append(c2)  # Add second child
            
            # Replace population with offspring (no elitism)
            population = offspring
        
        # If we reach here, this attempt failed
        total_generations += max_generations  # Update total generations
        fitnesses = [ga_fitness(x) for x in population]  # Calculate final fitnesses
        best_idx = max(range(len(fitnesses)), key=lambda i: fitnesses[i])  # Find best individual
        best_conflicts = total_conflicts(population[best_idx])  # Get best conflicts
        print(f"❌ Attempt {attempt} failed. Best conflicts: {best_conflicts}")
        print(f"🔄 Restarting with new seed...")
        
        # Continue to next attempt (infinite loop until solution found)


In [48]:
# =============================================================================
# MAIN SOLVER FUNCTION
# =============================================================================
# Central function that coordinates the entire Sudoku solving process

def solve_sudoku(puzzle_path, algorithm, group_id, puzzle_type):
    """
    Main function to solve Sudoku puzzle using specified algorithm
    
    Args:
        puzzle_path (str): Path to the puzzle file
        algorithm (str): Algorithm to use ('bt', 'sa', 'ga')
        group_id (str): Group identifier for output file naming
        puzzle_type (str): Puzzle difficulty type for output file naming
        
    Returns:
        tuple: (solution, decision_count, success_flag)
            - solution: Solved Sudoku object or None
            - decision_count: Number of decisions made
            - success_flag: Boolean indicating if solution was found and valid
    """
    # Print header information
    print(f"Solving Sudoku using {algorithm.upper()} algorithm")
    print(f"Puzzle: {puzzle_path}")
    print("-" * 50)
    
    # Reset decision counter to start fresh
    reset_decisions()
    
    # Read puzzle from file
    grid = read_puzzle_txt(puzzle_path)
    print("Puzzle loaded successfully")
    
    # Create Sudoku object from grid
    sud = sudoku_from_grid(grid)
    
    # Solve based on selected algorithm
    if algorithm == 'bt':  # Simple backtracking without forward checking
        solution = solve_bt_simple(sud, True, True, True)  # Use all heuristics, no forward checking
    elif algorithm == 'fc':  # Backtracking with forward checking
        solution = solve_bt(sud, True, True, True)  # Use all heuristics (MRV, DEG, LCV) with FC
    elif algorithm == 'ac3':  # Backtracking with AC3 preprocessing
        solution = solve_bt_with_ac3(sud, True, True, True)  # Use all heuristics with AC3
    elif algorithm == 'sa':  # Simulated Annealing algorithm
        solution = solve_sa(sud, SA_T0, SA_ALPHA, SA_MAX_STEPS)
    elif algorithm == 'ga':  # Genetic Algorithm
        solution = solve_ga(sud, GA_POP, GA_CROSSOVER_RATE, GA_MUTATION_RATE, 
                           GA_MAX_GENERATIONS, GA_TOURNAMENT_K)
    else:
        raise ValueError(f"Unknown algorithm: {algorithm}")
    
    # Check if solution was found
    if solution is not None:
        # Validate that the solution is actually correct
        is_solved_result = is_solved(solution)
        
        if is_solved_result:  # Solution is valid
            print(f"✓ Solution found - Decisions: {decisions}")
            
            # Generate output filename based on parameters
            puzzle_number = os.path.splitext(os.path.basename(puzzle_path))[0]
            output_filename = f"{group_id}_{algorithm}_{puzzle_type}_{puzzle_number}.txt"
            
            # Write solution to file
            write_puzzle_txt(output_filename, solution['grid'])
            print(f"Solution written to: {output_filename}")
            
            # Return successful result
            return solution, decisions, True
        else:  # Solution found but invalid
            print("✗ Solution found but validation failed!")
            return solution, decisions, False
    else:  # No solution found
        print("✗ No solution found")
        return None, decisions, False


In [49]:
# =============================================================================
# MAIN EXECUTION
# =============================================================================
# Entry point for running the Sudoku solver with configured parameters

if __name__ == "__main__":
    # Call the main solver function with configured parameters
    solution, decision_count, success = solve_sudoku(PUZZLE_PATH, ALGORITHM, GROUP_ID, PUZZLE_TYPE)
    
    # Print final results summary
    print("\n" + "=" * 50)
    print("FINAL RESULTS")
    print("=" * 50)
    print(f"Algorithm: {ALGORITHM.upper()}")  # Display algorithm used
    print(f"Puzzle: {PUZZLE_PATH}")  # Display puzzle file path
    print(f"Decisions: {decision_count}")  # Display number of decisions made
    print(f"Success: {success}")  # Display success status
    
    # If solution was found and is valid, display it
    if success and solution is not None:
        print("\nSolution:")  # Print solution header
        print(solution['grid'])  # Display the solved Sudoku grid


Solving Sudoku using GA algorithm
Puzzle: puzzles/Easy-P4.txt
--------------------------------------------------
Puzzle loaded successfully
🔄 GA Attempt 1 (seed 858)
GA Generation 0: best_conflicts=28
GA Generation 100: best_conflicts=6


KeyboardInterrupt: 

In [ ]:
# =============================================================================
# QUICK TEST FUNCTION
# =============================================================================
# Function for testing SA and GA algorithms with reduced parameters for faster execution

def quick_test():
    """
    Quick test with reduced parameters for faster algorithm testing
    
    Returns:
        tuple: (solution_sa, solution_ga) - results from both algorithms
    """
    print("Quick test of SA and GA algorithms...")
    
    # Use smaller parameters for quick testing (reduced from main parameters)
    test_sa_steps = 1000  # Reduced SA steps for faster testing
    test_ga_pop = 50  # Reduced GA population size
    test_ga_steps = 100  # Reduced GA generations
    
    # Load easy puzzle for testing
    grid = read_puzzle_txt("puzzles/Easy-P1.txt")
    sud = sudoku_from_grid(grid)
    
    # Test Simulated Annealing with reduced parameters
    print("Testing SA (quick)...")
    reset_decisions()  # Reset decision counter
    # Note: SA_TAU should be SA_ALPHA - this appears to be a typo in original code
    solution_sa = solve_sa(sud, SA_T0, SA_ALPHA, test_sa_steps)
    print(f"SA - Decisions: {decisions}, Conflicts: {total_conflicts(solution_sa)}, Solved: {is_solved(solution_sa)}")
    
    # Test Genetic Algorithm with reduced parameters
    print("\nTesting GA (quick)...")
    reset_decisions()  # Reset decision counter
    # Note: These parameters (GA_ELITE, GA_TOURNAMENT_Q, GA_PCROSSOVER, GA_PMUTATION) 
    # are not defined in the current code - this appears to be from an older version
    # Using current parameter names instead
    solution_ga = solve_ga(sud, test_ga_pop, GA_CROSSOVER_RATE, GA_MUTATION_RATE, 
                          test_ga_steps, GA_TOURNAMENT_K)
    print(f"GA - Decisions: {decisions}, Conflicts: {total_conflicts(solution_ga)}, Solved: {is_solved(solution_ga)}")
    
    return solution_sa, solution_ga  # Return both solutions

# Uncomment to run quick test
# quick_test()


In [ ]:
# =============================================================================
# MAIN EXECUTION (DUPLICATE)
# =============================================================================
# This appears to be a duplicate of the main execution block above
# Entry point for running the Sudoku solver with configured parameters

if __name__ == "__main__":
    # Call the main solver function with configured parameters
    solution, decisions, success = solve_sudoku(PUZZLE_PATH, ALGORITHM, GROUP_ID, PUZZLE_TYPE)
    
    # Print final results summary
    print("\n" + "=" * 50)
    print("FINAL RESULTS")
    print("=" * 50)
    print(f"Algorithm: {ALGORITHM.upper()}")  # Display algorithm used
    print(f"Puzzle: {PUZZLE_PATH}")  # Display puzzle file path
    print(f"Decisions: {decisions}")  # Display number of decisions made
    print(f"Success: {success}")  # Display success status
    
    # If solution was found and is valid, display it
    if success and solution is not None:
        print("\nSolution:")  # Print solution header
        print(solution)  # Display the solved Sudoku object (includes both grid and givens_mask)


Solving Sudoku using BT algorithm
Puzzle: puzzles/Easy-P4.txt
--------------------------------------------------
Puzzle loaded successfully
Trying variable (7, 0), values: [6]
Trying variable (0, 5), values: [1]
Trying variable (2, 5), values: [8]
Trying variable (3, 5), values: [6]
Trying variable (4, 4), values: [1]
Trying variable (4, 2), values: [6]
Trying variable (4, 8), values: [9]
Trying variable (4, 0), values: [8]
Trying variable (1, 0), values: [4]
Trying variable (0, 0), values: [5]
SOLUTION FOUND! Stopping at 45 decisions
✓ Solution found - Decisions: 45
Solution written to: 21_bt_easy_Easy-P4.txt

FINAL RESULTS
Algorithm: BT
Puzzle: puzzles/Easy-P4.txt
Decisions: 45
Success: True

Solution:
{'grid': array([[5, 6, 2, 4, 7, 1, 9, 8, 3],
       [4, 8, 7, 3, 2, 9, 5, 6, 1],
       [3, 1, 9, 6, 5, 8, 7, 4, 2],
       [2, 5, 3, 9, 8, 6, 1, 7, 4],
       [8, 7, 6, 2, 1, 4, 3, 5, 9],
       [9, 4, 1, 5, 3, 7, 8, 2, 6],
       [1, 9, 4, 8, 6, 5, 2, 3, 7],
       [6, 2, 5, 7, 9, 3,